Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

## **STATISTIK DESKRIPTIF**

1. Lakukan perhitungan central tendency (mean, median, dan modus).
2. Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
3. Lakukan pengecekan skewness dan kurtosis.
3. Analisa apakah ada data outlier berdasarkan sebaran datanya.
4. Cari tahu berapa persentase outlier terhadap data keseluruhan.
5. Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

## **API**

Buatlah API untuk:
- `Menampilkan` seluruh entry data setelah dilakukan handling outlier.
- `Menghapus` entry data, dari data yang telah dihandling outlier.

clue:
- Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary `df.to_dict()` atau json `df.to_json()` untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
- Pembuatan API dilakukan pada script `.py`
- Endpoint dan penamaan function pada API dibebaskan.


---



Data dapat diakses dari BigQuery dengan ketentuan:

- Project_id: `bigquery-public-data`
- Dataset: `san_francisco_bikeshare`
- Table: `bikeshare_trips`
- Ambil hanya kolom `duration_sec` saja
- Berikan LIMIT pada data yang diambil sebesar 3000 entry data

Koneksikan data di atas menggunakan `BigQuery` ke Google Colab dengan code berikut:
```py
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method `client.query('Masukkan Querynya').to_dataframe()`. Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

```py
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()
```



In [ ]:
# Working area

from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "dummy-467709" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

df = client.query('''
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3000
''').to_dataframe()

central tendency (mean, median, dan modus)

In [5]:
mean_val = df['duration_sec'].mean()
median_val = df['duration_sec'].median()
mode_val = df['duration_sec'].mode()[0]  # ambil modus pertama

print("Mean:", mean_val)
print("Median:", median_val)
print("Mode:", mode_val)

Mean: 800.4466666666667
Median: 497.0
Mode: 355


dispersi (standar deviasi, variance, dan range)

In [6]:
std_dev = df['duration_sec'].std()
variance_val = df['duration_sec'].var()
range_val = df['duration_sec'].max() - df['duration_sec'].min()

print("Std Dev:", std_dev)
print("Variance:", variance_val)
print("Range:", range_val)

Std Dev: 2894.7638269284507
Variance: 8379657.613693451
Range: 83106


Lakukan pengecekan skewness dan kurtosis.

In [7]:
skew_val = stats.skew(df['duration_sec'])
kurt_val = stats.kurtosis(df['duration_sec'])

print("Skewness:", skew_val)
print("Kurtosis:", kurt_val)

Skewness: 19.740009341206807
Kurtosis: 441.13623840372856


Analisa apakah ada data outlier berdasarkan sebaran datanya.

In [8]:
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['duration_sec'] < lower_bound) | (df['duration_sec'] > upper_bound)]
df_clean = df[(df['duration_sec'] >= lower_bound) & (df['duration_sec'] <= upper_bound)]

df_clean.to_csv("df_clean.csv", index=False)
print(f"Jumlah data awal: {len(df)}")
print(f"Jumlah data setelah outlier dihapus: {len(df_clean)}")
print(f"Jumlah outlier terdeteksi: {len(outliers)}")

Jumlah data awal: 3000
Jumlah data setelah outlier dihapus: 2826
Jumlah outlier terdeteksi: 174


Cari tahu berapa persentase outlier terhadap data keseluruhan.

In [10]:
outlier_percentage = (len(outliers) / len(df)) * 100

print(f"Jumlah data awal: {len(df)}")
print(f"Jumlah outlier: {len(outliers)}")
print(f"Persentase outlier: {outlier_percentage:.2f}%")

Jumlah data awal: 3000
Jumlah outlier: 174
Persentase outlier: 5.80%


Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?

In [11]:
mean_before = df['duration_sec'].mean()
median_before = df['duration_sec'].median()
mode_before = df['duration_sec'].mode()[0]

mean_after = df_clean['duration_sec'].mean()
median_after = df_clean['duration_sec'].median()
mode_after = df_clean['duration_sec'].mode()[0]

print(f"Mean sebelum: {mean_before:.2f}, sesudah: {mean_after:.2f}")
print(f"Median sebelum: {median_before:.2f}, sesudah: {median_after:.2f}")
print(f"Mode sebelum: {mode_before:.2f}, sesudah: {mode_after:.2f}")

Mean sebelum: 800.45, sesudah: 540.12
Median sebelum: 497.00, sesudah: 475.00
Mode sebelum: 355.00, sesudah: 355.00
